# Netflix Recommendation System

Netflix, kullanıcıların reklamsız film ve TV şovları izlemesine olanak tanıyan abonelik tabanlı bir yayın platformudur. Netflix'in popülerliğinin arkasındaki nedenlerden biri de öneri sistemidir. Öneri sistemi, kullanıcının ilgisine göre film ve TV şovları önerir. Bu projede biz de Netflix öneri sistemi oluşturacagiz.

<img src="https://artdogistanbul.com/wp-content/uploads/2021/04/493f5bba-81a4-11e9-bf79-066b49664af6_cm_1440w.png">

<a href="https://thecleverprogrammer.com/2022/07/05/netflix-recommendation-system-using-python/">Dataya Buradan Erişebilirsiniz</a>

In [10]:
import pandas as pd

pd.set_option("display.max_columns",100) #Satır ve sütunların hepsini göstermeye yarıyor
pd.set_option("display.max_rows",190) #isnull gibi degerlerin tamamını görmeye yarıyor

In [2]:
df=pd.read_csv("netflixData.csv")

In [3]:
df.head()

,Show Id,Title,Description,Director,Genres,Cast,Production Country,Release Date,Rating,Duration,Imdb Score,Content Type,Date Added
0,cc1b6ed9-cf9e-4057-8303-34577fb54477,(Un)Well,This docuseries takes a deep dive into the luc...,NaN,Reality TV,NaN,United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
1,e2ef4e91-fb25-42ab-b485-be8e3b23dedb,#Alive,"As a grisly virus rampages a city, a lone man ...",Cho Il,"Horror Movies, International Movies, Thrillers","Yoo Ah-in, Park Shin-hye",South Korea,2020.0,TV-MA,99 min,6.2/10,Movie,"September 8, 2020"
2,b01b73b7-81f6-47a7-86d8-acb63080d525,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...","Sabina Fedeli, Anna Migotto","Documentaries, International Movies","Helen Mirren, Gengher Gatti",Italy,2019.0,TV-14,95 min,6.4/10,Movie,"July 1, 2020"
3,b6611af0-f53c-4a08-9ffa-9716dc57eb9c,#blackAF,Kenya Barris and his family navigate relations...,NaN,TV Comedies,"Kenya Barris, Rashida Jones, Iman Benson, Genn...",United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
4,7f2d4170-bab8-4d75-adc2-197f7124c070,#cats_the_mewvie,This pawesome documentary explores how our fel...,Michael Margolis,"Documentaries, International Movies",NaN,Canada,2020.0,TV-14,90 min,5.1/10,Movie,"February 5, 2020"


In [4]:
df=df[["Title","Description","Genres","Imdb Score"]]

In [5]:
df.shape

(5967, 4)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5967 entries, 0 to 5966
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        5967 non-null   object
 1   Description  5967 non-null   object
 2   Genres       5967 non-null   object
 3   Imdb Score   5359 non-null   object
dtypes: object(4)
memory usage: 186.6+ KB


In [7]:
df.isnull().sum()

Title            0
Description      0
Genres           0
Imdb Score     608
dtype: int64

In [8]:
df=df.dropna()

In [11]:
df

,Title,Description,Genres,Imdb Score
0,(Un)Well,This docuseries takes a deep dive into the luc...,Reality TV,6.6/10
1,#Alive,"As a grisly virus rampages a city, a lone man ...","Horror Movies, International Movies, Thrillers",6.2/10
2,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...","Documentaries, International Movies",6.4/10
3,#blackAF,Kenya Barris and his family navigate relations...,TV Comedies,6.6/10
4,#cats_the_mewvie,This pawesome documentary explores how our fel...,"Documentaries, International Movies",5.1/10
...,...,...,...,...
5960,Zumbo's Just Desserts,Dessert wizard Adriano Zumbo looks for the nex...,"International TV Shows, Reality TV",7.1/10
5961,ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS,This documentary delves into the mystique behi...,"Documentaries, Music & Musicals",7.4/10
5962,الف مبروك,"On his wedding day, an arrogant, greedy accoun...","Comedies, Dramas, International Movies",7.4/10
5965,반드시 잡는다,After people in his town start turning up dead...,"Dramas, International Movies, Thrillers",6.5/10


<h4>tf/idf kullanrak tavsiye sistemi olusturma</h4>

In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
df['birlesim'] = df['Description'] + " " + df['Genres'] # Açıklama ve kategoriyi birleştirme

tf = TfidfVectorizer()
tfidf_matrix = tf.fit_transform(df['birlesim']) # TF-IDF Vektörizeri ile öznitelik matrisini oluşturma

kos = cosine_similarity(tfidf_matrix, tfidf_matrix) # Kosinüs benzerliği hesaplama

In [21]:
def recommend(title, cosine_sim=kos):
    # Film başlığının indeksini bul
    idx = df.index[df['Title'] == title].tolist()[0]

    # Benzerlik skorlarını al ve sıralama yap
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6]  # İlk film kendisidir --> En benzer 5 film
    movie_indices = [i[0] for i in sim_scores]

    return df['Title'].iloc[movie_indices]

In [25]:
recommend("Lucifer")

4149                          Selection Day
2447                                Justice
3903    ReMastered: Massacre at the Stadium
2784                            Lost Bullet
5494                                Trial 4
Name: Title, dtype: object

In [27]:
import pickle

In [32]:
with open('netflix.pkl', 'wb') as f:
    pickle.dump(tf, f)

In [31]:
df.to_csv("netflix_data.csv", index=False)  # Veriyi CSV olarak kaydet

Sonuc olarak netflixteki filmleri tf idf vectorizer kullanarak öneri sistemi olusturduk ve modelimiz kaydettik